In [0]:
# -------------------------------
# Logging setup (self-contained)
# -------------------------------

from pyspark.sql.types import *
from pyspark.sql.functions import current_timestamp
import uuid

LOG_PATH = "/FileStore/project/logs/pipeline_logs"

log_schema = StructType([
    StructField("run_id", StringType()),
    StructField("pipeline_layer", StringType()),
    StructField("notebook_name", StringType()),
    StructField("event_type", StringType()),   # START / END / ERROR / REJECTED
    StructField("record_count", LongType()),
    StructField("status", StringType()),       # RUNNING / SUCCESS / FAILED
    StructField("error_message", StringType()),
    StructField("event_timestamp", TimestampType())
])

# Create logging table if it does not exist
if not spark._jsparkSession.catalog().tableExists("delta.`/FileStore/project/logs/pipeline_logs`"):
    spark.createDataFrame([], log_schema) \
        .write.format("delta") \
        .mode("overwrite") \
        .save(LOG_PATH)

def log_event(layer, notebook, event_type, record_count, status, error_msg=None):

    row = [(
        str(uuid.uuid4()),
        layer,
        notebook,
        event_type,
        int(record_count),
        status,
        error_msg,          # can be None safely now
        None                # placeholder for timestamp
    )]

    df = spark.createDataFrame(row, schema=log_schema) \
              .withColumn("event_timestamp", current_timestamp())

    df.write.format("delta").mode("append").save(LOG_PATH)



In [0]:
# Task 5.1: Start MERGE logic

from delta.tables import DeltaTable

notebook_name = "silver_merge"
log_event("silver", notebook_name, "START", 0, "RUNNING")

silver_delta = DeltaTable.forPath(
    spark, "/FileStore/project/silver/sales"
)


In [0]:
# Task 5.2: Perform MERGE (UPSERT)

try:
    silver_delta.alias("t").merge(
        source=valid_sales.alias("s"),
        condition="t.transaction_id = s.transaction_id"
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()

    log_event("silver", notebook_name, "END",
              valid_sales.count(), "SUCCESS")

except Exception as e:
    log_event("silver", notebook_name, "ERROR", 0, "FAILED", str(e))
    raise


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4725766364173296>, line 5
      1 # Task 5.2: Perform MERGE (UPSERT)
      3 try:
      4     silver_delta.alias("t").merge(
----> 5         source=valid_sales.alias("s"),
      6         condition="t.transaction_id = s.transaction_id"
      7     ).whenMatchedUpdateAll() \
      8      .whenNotMatchedInsertAll() \
      9      .execute()
     11     log_event("silver", notebook_name, "END",
     12               valid_sales.count(), "SUCCESS")
     14 except Exception as e:

NameError: name 'valid_sales' is not defined